## 🔧 Preliminary Setup
Clarify the configuration and creates the folders

#### Specify Configuration File

In [43]:
# Reuse the same simple JSON used in the DEM/land notebook
settings_file = "danakil.json"  
print(f"Using settings file: {settings_file}")

Using settings file: danakil.json


In [44]:
#### Import libreries

In [45]:
import os, json, logging
import numpy as np
import pandas as pd
import rasterio as rio
from osgeo import gdal, gdalconst
from pathlib import Path
#from scipy import ndimage as nd


# ✅ Import the existing converter from libraries.io_tools (as in your other notebook)
from io_tools import convertAIIGrid
print("Loaded convertAIIGrid from libraries.io_tools")

from geo_tools import rasterize_vector_to_model, regrid_to_model
from io_tools import write_tif_from_grid

Loaded convertAIIGrid from libraries.io_tools


In [46]:
#### Create work folders

In [47]:
# Define the project root directory
project_root = str(Path().cwd()).replace('notebook','')

# Load configuration from the YAML file
with open(os.path.join(project_root, "settings", settings_file), 'r') as f:
    config = json.load(f)
    cfg = config.copy()
    
print(f"Configuration loaded from settings file: {settings_file}")


# Generate folder tree based on the configuration
path_settings = config['path']
for key in path_settings.keys():
    path_settings[key] = os.path.join(project_root, "projects", config['general']['project'], path_settings[key])

domain = config["general"]["domain"]
dem_path  = os.path.join(path_settings["output"], domain + ".dem.txt")

print("Domain:", domain)
print("Output:", path_settings["output"].replace("/home/continuumuser/workdir/",""))
print("Ancillary:", path_settings["ancillary"].replace("/home/continuumuser/workdir/",""))
print("DEM:", dem_path.replace("/home/continuumuser/workdir/",""))


Configuration loaded from settings file: danakil.json
Domain: danakil
Output: projects/training_eth/output/danakil
Ancillary: projects/training_eth/ancillary
DEM: projects/training_eth/output/danakil/danakil.dem.txt


#### Local dataset files (edit if needed)

In [49]:
LAND_SOIL_DIR = os.path.join(project_root, "sources/datasets/Land_Soil")
LC_MAP   = os.path.join(LAND_SOIL_DIR, "maps",  "C3S-ESACII-LC-ETH.tif")
SOIL_CONT_MAPS = {}
SOIL_CONT_MAPS["sand"] = os.path.join(LAND_SOIL_DIR, "maps",  "average_sand_ETH_cpr.tif")
SOIL_CONT_MAPS["clay"] = os.path.join(LAND_SOIL_DIR, "maps",  "average_clay_ETH_cpr.tif")
TABLE_CN = os.path.join(LAND_SOIL_DIR, "tables","ESA_CCI_to_CN.csv")
TABLE_VEG= os.path.join(LAND_SOIL_DIR, "tables","ESA_CCI_to_veg.csv")

for p in [LC_MAP, SOIL_CONT_MAPS["sand"], SOIL_CONT_MAPS["clay"], TABLE_CN, TABLE_VEG]:
    print(p.replace("/home/continuumuser/workdir/",""), "→", "OK" if os.path.exists(p) else "MISSING")

sources/datasets/Land_Soil/maps/C3S-ESACII-LC-ETH.tif → OK
sources/datasets/Land_Soil/maps/average_sand_ETH_cpr.tif → OK
sources/datasets/Land_Soil/maps/average_clay_ETH_cpr.tif → OK
sources/datasets/Land_Soil/tables/ESA_CCI_to_CN.csv → OK
sources/datasets/Land_Soil/tables/ESA_CCI_to_veg.csv → OK


## 🧩 Soil Tools
Declare a set of function for soil classification that will be used later

In [50]:
def fill(data, invalid=None):
    """Fill nodata/zero values using pure NumPy nearest neighbour."""
    if invalid is None:
        invalid = (np.isnan(data)) | (data == -9999) | (data == 0)
    valid_mask = ~invalid
    if not np.any(valid_mask):
        return data
    # Get coordinates of valid pixels
    valid_yx = np.argwhere(valid_mask)
    valid_vals = data[valid_mask]
    # Get coordinates of all pixels
    all_yx = np.indices(data.shape).reshape(2, -1).T
    filled_data = data.copy().reshape(-1)
    for idx, (y, x) in enumerate(all_yx):
        if invalid[y, x]:
            # Compute squared distances to valid pixels
            dist2 = (valid_yx[:, 0] - y)**2 + (valid_yx[:, 1] - x)**2
            nearest_idx = np.argmin(dist2)
            filled_data[idx] = valid_vals[nearest_idx]
    return filled_data.reshape(data.shape)


In [51]:
# Soil type classification
def classify_usda(out_map, sand, clay, silt):
    out_map = np.where((clay >= 0.4) & (sand <= 0.45) & (silt < 0.4), 1, out_map)   # clay
    out_map = np.where((clay >= 0.4) & (silt >= 0.4), 2, out_map)                   # silty-clay
    out_map = np.where((clay >= 0.27) & (clay < 0.4) & (sand <= 0.2), 3, out_map)   # silty-clay-loam
    out_map = np.where((clay >= 0.35) & (sand >= 0.45), 4, out_map)                 # sandy-clay
    out_map = np.where((clay >= 0.2) & (clay < 0.35) & (silt < 0.28) & (sand > 0.45), 5, out_map) # sandy-clay-loam
    out_map = np.where((clay >= 0.27) & (clay < 0.4) & (sand > 0.2) & (sand <= 0.45), 6, out_map) # clay-loam
    out_map = np.where((silt >= 0.8) & (clay < 0.12), 7, out_map)                    # silt
    out_map = np.where(((silt >= 0.5) & (clay >= 0.12) & (clay < 0.27)) | ((silt >= 0.5) & (silt < 0.8) & (clay < 0.12)), 8, out_map) # silt-loam
    out_map = np.where((clay >= 0.07) & (clay <= 0.27) & (silt >= 0.28) & (silt < 0.5) & (sand <= 0.52), 9, out_map) # loam
    out_map = np.where((silt + 1.5 * clay) < 0.15, 10, out_map)                      # sand
    out_map = np.where(((silt + 1.5 * clay) >= 0.15) & ((silt + 2 * clay) < 0.3), 11, out_map) # loamy-sand
    out_map = np.where(((clay >= 0.07) & (clay <= 0.2) & (sand > 0.52) & ((silt + 2 * clay) >= 0.3)) |
                       ((clay < 0.07) & (silt < 0.5) & ((silt + 2 * clay) >= 0.3)), 12, out_map) # sandy-loam
    out_map = np.where(sand == 0, -9999, out_map)  # guard for nodata
    return out_map

# Hydrological soil group identification
def classify_hsg(out_map, soil_class):
    out_map = np.where(soil_class == 10, 1, out_map)   # A
    out_map = np.where((soil_class == 11) | (soil_class == 12), 2, out_map)  # B
    out_map = np.where(((soil_class > 4) & (soil_class < 10)) | (soil_class == 3), 3, out_map)  # C
    out_map = np.where((soil_class == 1) | (soil_class == 2) | (soil_class == 4), 4, out_map)   # D
    return out_map

## 🗺️ Regrid maps on model grid
#### Read reference grid from the model DEM

In [52]:
from osgeo import osr

def epsg_to_wkt(epsg_code: str) -> str:
    s = osr.SpatialReference()
    s.ImportFromEPSG(int(epsg_code.split(":")[1]))
    return s.ExportToWkt()

match_ds = gdal.Open(dem_path, gdalconst.GA_ReadOnly)
out_profile = rio.open(dem_path).profile
out_profile["driver"] = "GTiff"

dem = np.array(match_ds.GetRasterBand(1).ReadAsArray())
geotrans = match_ds.GetGeoTransform()
wide = match_ds.RasterXSize
high = match_ds.RasterYSize

proj_wkt_dem = match_ds.GetProjection() 
proj_wkt = proj_wkt_dem.strip() if proj_wkt_dem and proj_wkt_dem.strip() else epsg_to_wkt("EPSG:4326")

# bbox DEM
lon_min = min(geotrans[0], geotrans[0] + wide * geotrans[1])
lon_max = max(geotrans[0], geotrans[0] + wide * geotrans[1])
lat_min = min(geotrans[3], geotrans[3] + high * geotrans[5])
lat_max = max(geotrans[3], geotrans[3] + high * geotrans[5])

model_srs = {
    "proj": proj_wkt,  
    "geotrans": geotrans,
    "wide": wide,
    "high": high,
    "bbox": [lon_min, lon_max, lat_min, lat_max], 
}
print("Grid:", wide, "x", high)

Grid: 640 x 734


#### Regrid Local Maps (ESA CCI, sand, clay)

In [53]:
anc_lc   = os.path.join(path_settings["ancillary"], "LULC_regrid.tif")
anc_sand = os.path.join(path_settings["ancillary"], "average_regrid_sand.tif")
anc_clay = os.path.join(path_settings["ancillary"], "average_regrid_clay.tif")

#src = gdal.Open(LC_MAP); src_wkt = src.GetProjection()
#write_tif_from_grid(src_wkt, model_srs, LC_MAP, anc_lc, gdalconst.GRA_Mode, file_type=gdalconst.GDT_Float32)

#src = gdal.Open(SAND_MAP); src_wkt = src.GetProjection()
#write_tif_from_grid(src_wkt, model_srs, SAND_MAP, anc_sand, gdalconst.GRA_Bilinear, file_type=gdalconst.GDT_Float32)

#src = gdal.Open(CLAY_MAP); src_wkt = src.GetProjection()
#write_tif_from_grid(src_wkt, model_srs, CLAY_MAP, anc_clay, gdalconst.GRA_Bilinear, file_type=gdalconst.GDT_Float32)

#print("Regrid done →", os.path.basename(anc_lc), os.path.basename(anc_sand), os.path.basename(anc_clay))

regrid_to_model(LC_MAP,   anc_lc,   model_srs, resample_alg=gdalconst.GRA_Mode,     dst_nodata=-9999.0)
regrid_to_model(SOIL_CONT_MAPS["sand"], anc_sand, model_srs, resample_alg=gdalconst.GRA_Bilinear, dst_nodata=-9999.0)
regrid_to_model(SOIL_CONT_MAPS["clay"], anc_clay, model_srs, resample_alg=gdalconst.GRA_Bilinear, dst_nodata=-9999.0)

print("Regrid done →", os.path.basename(anc_lc), os.path.basename(anc_sand), os.path.basename(anc_clay))


Regrid done → LULC_regrid.tif average_regrid_sand.tif average_regrid_clay.tif


#### Read Lake mask (if used)

In [54]:
# Lake mask (raster o vettore). Flag in cfg.flags o cfg.algorithm.flags
if "algorithm" in cfg and "flags" in cfg["algorithm"]:
    use_lake_mask = bool(cfg["algorithm"]["flags"].get("use_lake_mask", False))
else:
    use_lake_mask = bool(cfg.get("flags", {}).get("use_lake_mask", False))

lake_entry = cfg.get("data", {}).get("lake_mask", None)
lake_mask = None

if not use_lake_mask or not lake_entry:
    print("Lake mask not used.")
else:
    lake_path = lake_entry if os.path.isabs(lake_entry) else os.path.join(path_settings["data"], lake_entry)
    if not os.path.exists(lake_path):
        print(f"Lake mask not found: {lake_path}")
    else:
        ext = os.path.splitext(lake_path)[1].lower()
        is_vector = ext in [".shp", ".gpkg", ".geojson", ".json"]
        tmp_lake = os.path.join(path_settings["ancillary"], "lake_mask_regrid.tif")
        if is_vector:
            rasterize_vector_to_model(lake_path, tmp_lake, model_srs, burn_value=1, dst_nodata=0, assumed_src_epsg="EPSG:4326")
        else:
            regrid_to_model(lake_path, tmp_lake, model_srs, resample_alg=gdalconst.GRA_NearestNeighbour, dst_nodata=0)
        lake_mask = rio.open(tmp_lake).read(1).astype(int)
        lake_mask = (lake_mask != 0).astype(int)
        print("Lake mask ready:", tmp_lake)


Lake mask ready: /home/continuumuser/workdir/projects/training_eth/ancillary/lake_mask_regrid.tif


## 🧮 Calculate soil maps

In [55]:
#### Soil volume and CN

In [56]:
DEFAULT_SAND = 40.0
DEFAULT_CLAY = 20.0

# Sand
sand_map_raw = rio.open(anc_sand).read(1).astype(np.float32)
sand_map = np.where((sand_map_raw == -9999) | (sand_map_raw == 0) | np.isnan(sand_map_raw),
                    DEFAULT_SAND, sand_map_raw)/10

# Clay
clay_map_raw = rio.open(anc_clay).read(1).astype(np.float32)
clay_map = np.where((clay_map_raw == -9999) | (clay_map_raw == 0) | np.isnan(clay_map_raw),
                    DEFAULT_CLAY, clay_map_raw)/10

# Silt
silt = 100.0 - sand_map - clay_map

# USDA soil classification
soil_class = -9999 * np.ones_like(sand_map, dtype=np.float32)
soil_class = classify_usda(soil_class, sand_map/100.0, clay_map/100.0, silt/100.0)
soil_class[dem < -9000] = -9999

# Hydrological Soil Group classification
hsg = -9999 * np.ones_like(dem, dtype=np.float32)
hsg = classify_hsg(hsg, soil_class)

# CN lookup by ESA class + HSG using NumPy lookup table
lulc_df = pd.read_csv(TABLE_CN, header=0, usecols=["ID","A","B","C","D"], index_col="ID", sep=',')
max_id = int(lulc_df.index.max())
lut = np.full((max_id + 1, 4), -9999, dtype=np.float32)
lut[lulc_df.index.values] = lulc_df[["A", "B", "C", "D"]].values

lulc_map = rio.open(anc_lc).read(1).astype(np.int32)

cn_map = np.full_like(dem, -9999, dtype=np.float32)
for num, col_idx in enumerate([0, 1, 2, 3], start=1):
    vals = lut[lulc_map, col_idx]
    cn_map = np.where(hsg == num, vals, cn_map)

# Compute S only for valid CN values
valid_cn = (cn_map > 0) & (cn_map <= 100)
S = np.full_like(cn_map, -9999, dtype=np.float32)
S[valid_cn] = 254.0 * ((100.0 / cn_map[valid_cn]) - 1.0)

# Apply lake mask to CN and S
cn_map = np.where(lake_mask == 1, 5, cn_map)
S = np.where(lake_mask == 1, 4500, S)

# Temp GeoTIFFs
t_cn = os.path.join(path_settings["ancillary"], "temp_cn.tif")
t_S  = os.path.join(path_settings["ancillary"], "temp_S.tif")
with rio.open(t_cn, 'w', **out_profile) as dst:
    dst.write(cn_map, 1)
with rio.open(t_S, 'w', **out_profile) as dst:
    dst.write(S, 1)

# Export to ASCII grid
convertAIIGrid(t_cn, os.path.join(path_settings["output"], f"{domain}.cn.txt"), outType="Int16", precision=0)
convertAIIGrid(t_S, os.path.join(path_settings["output"], f"{domain}.soil_vmax.txt"), outType="Float32", precision=2)

print("CN & S exported to:", path_settings["output"].replace("/home/continuumuser/workdir/", ""))


CN & S exported to: projects/training_eth/output/danakil


#### Soil Hydraulic Properties (ct, infilt, soil_ksat_drain)

In [57]:
# Saxton et al. (1986) PTFs using sand/clay (%)
sand = sand_map.astype(np.float32)
clay = clay_map.astype(np.float32)

a = np.exp(-4.396 - 0.0715*clay - 0.000488*(sand**2) - 0.00004285*(sand**2)*clay)
b = -3.14 - 0.00222*(clay**2) - 0.00003484*(sand**2)*clay
ct = (0.33333/a)**(1.0/b)  # field capacity (theta at 1/3 bar)

porosity = 0.332 - 0.0007251*sand + 0.1276*np.log10(np.maximum(clay, 1e-6))
ks_infilt = np.exp((12.012 - 0.0755*sand) + (-3.895 + 0.03671*sand - 0.1103*clay + 0.00087546*(clay**2)) / np.maximum(porosity, 1e-6))
ks_drain = ks_infilt.copy()

# Apply lake mask overrides if present
if 'lake_mask' in globals() and isinstance(lake_mask, np.ndarray) and use_lake_mask:
    ks_infilt = np.where(lake_mask == 1, 100.0, ks_infilt)
    ks_drain  = np.where(lake_mask == 1, 0.03, ks_drain)
    ct        = np.where(lake_mask == 1, 0.9, ct)

# Save temp tifs
t_ct = os.path.join(path_settings["ancillary"], "temp_ct.tif")
t_k1 = os.path.join(path_settings["ancillary"], "temp_ksat_infilt.tif")
t_k2 = os.path.join(path_settings["ancillary"], "temp_ksat_drain.tif")
with rio.open(t_ct, 'w', **out_profile) as dst: dst.write(ct.astype(np.float32), 1)
with rio.open(t_k1, 'w', **out_profile) as dst: dst.write(ks_infilt.astype(np.float32), 1)
with rio.open(t_k2, 'w', **out_profile) as dst: dst.write(ks_drain.astype(np.float32), 1)

# Export AAIGrid
convertAIIGrid(t_ct, os.path.join(path_settings["output"], f"{domain}.ct.txt"), outType="Float32", precision=2)
convertAIIGrid(t_k1, os.path.join(path_settings["output"], f"{domain}.soil_ksat_infilt.txt"), outType="Float32", precision=2)
convertAIIGrid(t_k2, os.path.join(path_settings["output"], f"{domain}.soil_ksat_drain.txt"), outType="Float32", precision=2)

print("ct, soil_ksat_infilt, soil_ksat_drain exported to:", path_settings["output"].replace("/home/continuumuser/workdir/",""))


ct, soil_ksat_infilt, soil_ksat_drain exported to: projects/training_eth/output/danakil


## 🌱 Vegetation Layers (BareSoil, RSmin, Hveg, Gd)

In [58]:
lulc_to_veg = pd.read_csv(TABLE_VEG, header=0, usecols=["ID","BareSoil","RSmin","Hveg","Gd"], index_col="ID", sep=',')
lulc = rio.open(anc_lc).read(1).astype(np.int32)

for var, outType, precision in [
    ("BareSoil", "Int16",   0),
    ("RSmin",    "Float32", 2),
    ("Hveg",     "Float32", 2),
    ("Gd",       "Float32", 2),
]:
    arr = lulc_to_veg.reindex(lulc.flatten())[var].fillna(-9999).values.reshape(dem.shape).astype(np.float32)
    arr[dem < -9000] = -9999
    t_var = os.path.join(path_settings["ancillary"], f"temp_{var}.tif")
    with rio.open(t_var, 'w', **out_profile) as dst: dst.write(arr, 1)
    convertAIIGrid(t_var, os.path.join(path_settings["output"], f"{domain}.{var}.txt"), outType=outType, precision=precision)

print("Vegetation maps exported to:", path_settings["output"].replace("/home/continuumuser/workdir/",""))


Vegetation maps exported to: projects/training_eth/output/danakil


#### Cleanup

In [59]:
# Remove auxiliary files some GDAL versions might leave
os.system(f'find "{path_settings["output"]}" -maxdepth 1 -type f -name "*.xml" -delete || true')
print("Cleanup done.")

Cleanup done.
